# EXAMEN DE STATISTIQUE EXPLORATOIRE SPATIALE - ENSAE - ISE1CL - 2025\2026
**Eleve :** David NGUEAJIO

##  PARTIE 2 ECMWF

Le but de cette partie est de mobiliser les outils de *statistiques exploratoires spatiale* pour analyser la structure spatiale des **précipitations** au **Ghana** à partir de données climatiques issue de sources satellitaires et de réanalyses. Avant toute analyse, commençons par appeler les bibliothèques de travail.

## SECTION 1: Accès et préparations des données

Nous commençons en initialisant l'API de GEE

In [1]:

import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()

Enter verification code:  4/1ASc3gC3vG_OPNUdHCe6l1ZyYo5UeHdu4qUmXmK2bXlPp4o5XvZNaradzfGM



Successfully saved authorization token.


In [2]:
ee.Initialize(project="ee-davidngueajio")

print("Earth Engine est connecté :", ee.Number(1).getInfo())

Earth Engine est connecté : 1


Maintenant, nous allons charger la collection avec les données sur la structure spatiale des précipitations journalières au Ghana.

In [31]:
# ASSET = "projects/ee-davidngueajio/assets/"
ECMWF = ee.ImageCollection("ECMWF/ERA5/DAILY")  #Nous chargeons la collection

Le chargement étant effectué, inspectons là pour identifier des informations liées à la bande que nous recherchons.

In [32]:
print('Quelques informations :', ECMWF.first().getInfo())

Quelques informations : {'type': 'Image', 'bands': [{'id': 'mean_2m_air_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}, {'id': 'minimum_2m_air_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}, {'id': 'maximum_2m_air_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}, {'id': 'dewpoint_2m_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}, {'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}, {'id':

La bande que nous recherchons est donc bien présente.

{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}

Maintenant, selectionnons la bande total_precipitation

In [45]:
daily_rain_raw = ECMWF.select('total_precipitation')

A présent, exploitons la collection GADM pour définir la géométrie du pays.

In [43]:
# Charger Ghana depuis GADM
ghana = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))

ghana_geometry = ghana.first().geometry()

ghana.getInfo().keys()
ghana_geometry.getInfo().keys()

dict_keys(['type', 'coordinates'])

Maintenant nous allons nous restreindre à la période 2015-2020

In [47]:
daily_rain = daily_rain_raw.filterDate('2015-01-01', '2020-12-31').filterBounds(ghana)